Pulling data using coinbase api rather than yfinance like in other files

In [1]:
import numpy as np
import pandas as pd
import requests as rq
import time, datetime
import json

#https://docs.cloud.coinbase.com/exchange/docs/welcome

In [2]:
#Fetch data length
url = 'https://api.exchange.coinbase.com/products/{}/candles?granularity=86400'

#Fetch dataset with given time interval
url2 = 'https://api.exchange.coinbase.com/products/{}/candles?granularity=86400&start={}&end={}'

In [3]:
ticker = 'BTC-USD'

In [4]:
#Calculates time length in each batch
def delta(u):
    resp = rq.get(u.format(ticker)).json()
    dt = resp[0][0] = resp[-1][0]
    return dt

In [5]:
#Convert numeric timestamp to ISO time format to be sent to Coinbase
def timestamp(x, conv='%Y-%m-%dT%H:%M:%SZ'):
    return datetime.datetime.fromtimestamp(x).strftime(conv)

In [6]:
#Batch seconds
dT = delta(url)

#Initial timestamps
t1 = int(time.time())
t0 = t1 - dT

#Number of batches we wish to fetch
years = 6

#Table too hold data
table = []

In [10]:
#For loop too get all info
for t in range(years):
    #Show how much is left to pull
    print(years - t, 'pulling')
    
    #Set up ISO timestamps
    T0, T1 = timestamp(t0), timestamp(t1)
    
    #Building table
    table = table + rq.get(url2.format(ticker, T0, T1)).json()
    
    #Sleep timer too not get kicked
    time.sleep(1)
    
    #Subtracting timestamps
    t0 = t0 - dT
    t1 = t1 - dT
    
# Creating dataframe w/necessary columns
dataframe = pd.DataFrame(table, columns=['Time','Low','High','Open','Close','Volume'])

# Changing Time section to show actual date rather than numbers
time_values = dataframe['Time'].values
time_values = [timestamp(t, conv='%m-%d-%Y') for t in time_values]
dataframe['Time'] = time_values

print(dataframe)

6 pulling
5 pulling
4 pulling
3 pulling
2 pulling
1 pulling
            Time       Low      High      Open     Close        Volume
0     12-12-2023  40553.15  43475.00  41477.60  42885.54  15796.881714
1     12-11-2023  40636.51  42088.56  41229.61  41477.39  13414.867737
2     12-10-2023  40150.00  43811.81  43793.67  41229.61  25524.670947
3     12-09-2023  43562.06  44049.51  43722.84  43793.68   3919.094236
4     12-08-2023  43586.14  44384.68  44192.86  43723.25   5469.407084
...          ...       ...       ...       ...       ...           ...
3064  07-23-2015    276.43    291.52    277.23    289.12   7362.469083
3065  07-22-2015    276.28    279.75    277.96    277.39   5306.919575
3066  07-21-2015    275.01    278.54    277.33    277.89   4687.909383
3067  07-20-2015    276.85    281.27    279.96    277.32   4943.559434
3068  07-19-2015    277.37    280.00    277.98    280.00    782.883420

[3069 rows x 6 columns]


In [11]:
dataframe.head()

,Time,Low,High,Open,Close,Volume
0,12-12-2023,40553.15,43475.00,41477.60,42885.54,15796.881714
1,12-11-2023,40636.51,42088.56,41229.61,41477.39,13414.867737
2,12-10-2023,40150.00,43811.81,43793.67,41229.61,25524.670947
3,12-09-2023,43562.06,44049.51,43722.84,43793.68,3919.094236
4,12-08-2023,43586.14,44384.68,44192.86,43723.25,5469.407084


In [12]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3069 entries, 0 to 3068
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    3069 non-null   object 
 1   Low     3069 non-null   float64
 2   High    3069 non-null   float64
 3   Open    3069 non-null   float64
 4   Close   3069 non-null   float64
 5   Volume  3069 non-null   float64
dtypes: float64(5), object(1)
memory usage: 144.0+ KB


In [13]:
dataframe.isnull().sum()

Time      0
Low       0
High      0
Open      0
Close     0
Volume    0
dtype: int64